In [ ]:
!pip install flask pyngrok huggingface_hub flask-cors

In [ ]:
import tensorflow as tf

# Check for GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Set memory growth so it doesn’t allocate all GPU memory at once
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU detected and memory growth set: {gpus}")
    except RuntimeError as e:
        print(f"⚠️ GPU setup error: {e}")
else:
    print("❌ No GPU detected. Using CPU instead.")

✅ GPU detected and memory growth set: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# from flask import Flask, request, jsonify
# from flask_cors import CORS
# import cv2
# import numpy as np
# from tensorflow.keras.models import load_model
# from pyngrok import ngrok
# from huggingface_hub import hf_hub_download
# import threading
# import os

# app = Flask(__name__)
# CORS(app)  # Enable CORS for all routes

# ngrok.set_auth_token("2aoX3HVyAg2zwx9LJ3n8kfI5TQo_2Lc5vP5tnTpidns62pRdc")

# # Download the model
# repo_id = "andrewatef/violence-detection001"
# filename = "violence-detection001.keras"
# model_path = hf_hub_download(repo_id=repo_id, filename=filename, local_dir="/content")

# # Model parameters
# IMAGE_HEIGHT, IMAGE_WIDTH = 224, 224
# CLASSES_LIST = ["NonViolence", "Violence"]
# SEQUENCE_LENGTH = 12

# # Load the model
# MoBiLSTM_model = load_model(model_path)
# print("Model loaded successfully!")

# def predict_video(video_file_path, sequence_length):
#     video_reader = cv2.VideoCapture(video_file_path)

#     frames_list = []
#     video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
#     skip_frames_window = max(int(video_frames_count/sequence_length), 1)

#     for frame_counter in range(sequence_length):
#         video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
#         success, frame = video_reader.read()

#         if not success:
#             break

#         resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
#         normalized_frame = resized_frame / 255
#         frames_list.append(normalized_frame)

#     predicted_labels_probabilities = MoBiLSTM_model.predict(np.expand_dims(frames_list, axis=0))[0]
#     predicted_label = np.argmax(predicted_labels_probabilities)
#     predicted_class_name = CLASSES_LIST[predicted_label]
#     confidence = float(predicted_labels_probabilities[predicted_label])

#     video_reader.release()
#     return predicted_class_name, confidence

# @app.route('/predict', methods=['POST'])
# def predict():
#     try:
#         if 'video' not in request.files:
#             return jsonify({'error': 'No video file provided'}), 400

#         video_file = request.files['video']
#         temp_video_path = 'temp_video.mp4'
#         video_file.save(temp_video_path)

#         predicted_class, confidence = predict_video(temp_video_path, SEQUENCE_LENGTH)
#         os.remove(temp_video_path)

#         return jsonify({
#             'prediction': predicted_class,
#             'confidence': confidence
#         })

#     except Exception as e:
#         return jsonify({'error': str(e)}), 500

# def run_flask():
#     app.run(port=5000)

# if __name__ == '__main__':
#     try:
#         flask_thread = threading.Thread(target=run_flask)
#         flask_thread.daemon = True
#         flask_thread.start()

#         public_url = ngrok.connect(5000)
#         print(f"Public URL: {public_url}")
#         print("Send a POST request to {public_url}/predict with a video file")

#     except Exception as e:
#         print(f"An error occurred: {e}")

Exception ignored in: <function _xla_gc_callback at 0x7d6a79c7fba0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from pyngrok import ngrok
from huggingface_hub import hf_hub_download
import threading
import os
import uuid  # Added for unique filename generation

app = Flask(__name__)
CORS(app)

ngrok.set_auth_token("27IN8YZ6hTfAjARwQLgVMCqPwd7_7rpcSEhGfNfemyPVhzkNU")

# Download the model
repo_id = "andrewatef/violence-detection001"
filename = "violence-detection001.keras"
model_path = hf_hub_download(repo_id=repo_id, filename=filename, local_dir="/content")

# Model parameters
IMAGE_HEIGHT, IMAGE_WIDTH = 224, 224
CLASSES_LIST = ["NonViolence", "Violence"]
SEQUENCE_LENGTH = 12

# Load the model
MoBiLSTM_model = load_model(model_path)
print("Model loaded successfully!")

def predict_video(video_file_path, sequence_length):
    video_reader = cv2.VideoCapture(video_file_path)

    frames_list = []
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames_window = max(int(video_frames_count/sequence_length), 1)

    for frame_counter in range(sequence_length):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        success, frame = video_reader.read()

        if not success:
            break

        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        normalized_frame = resized_frame / 255
        frames_list.append(normalized_frame)

    predicted_labels_probabilities = MoBiLSTM_model.predict(np.expand_dims(frames_list, axis=0))[0]
    predicted_label = np.argmax(predicted_labels_probabilities)
    predicted_class_name = CLASSES_LIST[predicted_label]
    confidence = float(predicted_labels_probabilities[predicted_label])

    video_reader.release()
    return predicted_class_name, confidence

@app.route('/predict', methods=['POST'])
def predict():
    try:
        if 'video' not in request.files:
            return jsonify({'error': 'No video file provided'}), 400

        video_file = request.files['video']
        # Generate unique filename using UUID
        unique_id = str(uuid.uuid4())
        temp_video_path = f'temp_video_{unique_id}.mp4'

        try:
            video_file.save(temp_video_path)
            predicted_class, confidence = predict_video(temp_video_path, SEQUENCE_LENGTH)

            return jsonify({
                'prediction': predicted_class,
                'confidence': confidence
            })
        finally:
            # Ensure cleanup even if prediction fails
            if os.path.exists(temp_video_path):
                os.remove(temp_video_path)

    except Exception as e:
        return jsonify({'error': str(e)}), 500

def run_flask():
    app.run(port=5000)

if __name__ == '__main__':
    try:
        flask_thread = threading.Thread(target=run_flask)
        flask_thread.daemon = True
        flask_thread.start()

        public_url = ngrok.connect(5000)
        print(f"Public URL: {public_url}")
        print("Send a POST request to {public_url}/predict with a video file")

    except Exception as e:
        print(f"An error occurred: {e}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


violence-detection001.keras:   0%|          | 0.00/203M [00:00<?, ?B/s]

Model loaded successfully!
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Public URL: NgrokTunnel: "https://884b-34-82-193-33.ngrok-free.app" -> "http://localhost:5000"
Send a POST request to {public_url}/predict with a video file
